In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

In [3]:
x = np.random.poisson(5,10)
y = np.random.poisson(5,10)

### Многомерное нормальное распределение

Вектор размерности $d$ X = (X_1, ..., X_d) имеет многомерное нормальное распределение, если любая линейная комбинация его компонет имеет нормальное распределение. Плотность распределения имеет вид:

$$ p(x) = \frac{1}{\sqrt{2\pi}\,det\Sigma} exp \left( -\frac{1}{2}(x-\mu)^T\Sigma^{-1}(x-\mu)\right).$$

Матрица ковариации:
$$ \Sigma = E\left[(X-\mu)(X-\mu)^T \right].$$

** Расстояние Махаланобиса**:

$$D_M (x)= \sqrt{(x-\mu)^TS^{-1}(x-\mu)}.$$

Оценим матрицу ковариации для переменных:

In [4]:
covariance_xy = np.cov(x,y, rowvar=0)
inv_covariance_xy = np.linalg.inv(covariance_xy)

Центрируем данные:

In [5]:
xy_mean = np.mean(x),np.mean(y)
x_diff = np.array([x_i - xy_mean[0] for x_i in x])
y_diff = np.array([y_i - xy_mean[1] for y_i in y])

In [7]:
diff_xy = np.transpose([x_diff, y_diff])
diff_xy.shape, diff_xy

((10, 2), array([[-0.1, -0.2],
        [-1.1,  0.8],
        [-0.1, -0.2],
        [-0.1,  0.8],
        [-2.1, -1.2],
        [ 0.9,  1.8],
        [-1.1, -1.2],
        [ 1.9, -3.2],
        [-0.1,  1.8],
        [ 1.9,  0.8]]))

Рассмотрим расстояние до центра координат:

In [8]:
md = []
for i in range(len(diff_xy)):
    md.append(np.sqrt(np.dot(np.dot(np.transpose(diff_xy[i]),inv_covariance_xy),diff_xy[i])))
md

[0.15143810309081371,
 0.99389059801387825,
 0.15143810309081371,
 0.52138840320641511,
 1.8144849132221383,
 1.362942927817324,
 1.1600882930757941,
 2.5258637275037632,
 1.1636989096276102,
 1.5698764662637554]

То же самое, упакованное в одну функцию:

In [9]:
def MahalanobisDist(x, y):
    covariance_xy = np.cov(x,y, rowvar=0)
    inv_covariance_xy = np.linalg.inv(covariance_xy)
    xy_mean = np.mean(x),np.mean(y)
    x_diff = np.array([x_i - xy_mean[0] for x_i in x])
    y_diff = np.array([y_i - xy_mean[1] for y_i in y])
    diff_xy = np.transpose([x_diff, y_diff])
    
    md = []
    for i in range(len(diff_xy)):
        md.append(np.sqrt(np.dot(np.dot(np.transpose(diff_xy[i]),inv_covariance_xy),diff_xy[i])))
    return md

In [10]:
MahalanobisDist(x,y)

[0.15143810309081371,
 0.99389059801387825,
 0.15143810309081371,
 0.52138840320641511,
 1.8144849132221383,
 1.362942927817324,
 1.1600882930757941,
 2.5258637275037632,
 1.1636989096276102,
 1.5698764662637554]

По аналогии, с тем, что мы делали ранее для одномерных данным, можем "записать" в выбросы все наблюдения, которые отстоят достаточно далеко от центра координат:

In [15]:
def MD_removeOutliers(x, y):
    MD = MahalanobisDist(x, y)
    threshold = np.mean(MD) * 1.65 
    nx, ny, outliers = [], [], []
    for i in range(len(MD)):
        if MD[i] <= threshold:
            nx.append(x[i])
            ny.append(y[i])
        else:
            outliers.append(i)
    return (np.array(nx), np.array(ny), np.array(outliers))

In [16]:
print 'x:', x
print 'y:', y
MD_removeOutliers(x,y)

x: [4 3 4 4 2 5 3 6 4 6]
y: [4 5 4 5 3 6 3 1 6 5]


(array([4, 3, 4, 4, 2, 5, 3, 4, 6]),
 array([4, 5, 4, 5, 3, 6, 3, 6, 5]),
 array([7]))